# Бэггинг и случайный лес

Загрузите датасет digits с помощью функции load_digits из sklearn.datasets и подготовьте матрицу признаков X и ответы на обучающей выборке y (вам потребуются поля data и target в объекте, который возвращает load_digits).

In [2]:
from sklearn import datasets

In [4]:
raw_data = datasets.load_digits()

x = raw_data["data"]
y = raw_data["target"]

In [6]:
x.shape

(1797, 64)

In [7]:
y.shape

(1797,)

Для оценки качества далее нужно будет использовать cross_val_score из sklearn.cross_validation с параметром cv=10. Эта функция реализует k-fold cross validation c k равным значению параметра cv. Мы предлагаем использовать k=10, чтобы полученные оценки качества имели небольшой разброс, и было проще проверить полученные ответы. На практике же часто хватает и k=5. Функция cross_val_score будет возвращать numpy.ndarray, в котором будет k чисел - качество в каждом из k экспериментов k-fold cross validation. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод .mean() у массива, который возвращает cross_val_score.

In [8]:
cv = 10

С небольшой вероятностью вы можете натолкнуться на случай, когда полученное вами качество в каком-то из пунктов не попадет в диапазон, заданный для правильных ответов - в этом случае попробуйте перезапустить ячейку с cross_val_score несколько раз и выбрать наиболее «типичное» значение. Если это не помогает, то где-то была допущена ошибка.

Если вам захочется ускорить вычисление cross_val_score - можете попробовать использовать параметр n_jobs, но будьте осторожны: в одной из старых версий sklearn была ошибка, которая приводила к неверному результату работы cross_val_score при задании n_jobs отличным от 1. Сейчас такой проблемы возникнуть не должно, но проверить, что все в порядке, не будет лишним.

# Задание 1

Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. Эта величина и будет ответом в пункте 1.

In [13]:
from sklearn import tree
from sklearn import model_selection

clf_tree = tree.DecisionTreeClassifier()
rez = model_selection.cross_val_score(clf_tree, x, y, cv=cv)
rez

array([0.82162162, 0.89617486, 0.82320442, 0.8       , 0.79888268,
       0.87150838, 0.86592179, 0.84831461, 0.81355932, 0.79545455])

In [14]:
rez.mean()

0.833464222829113

In [15]:
with open("task1.txt", "w") as file:
    file.write(str(rez.mean()))

# Задание 2

Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100.

In [16]:
from sklearn import ensemble

clf_bag = ensemble.BaggingClassifier(clf_tree, n_estimators=100)

Качество классификации новой модели - ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [17]:
rez = model_selection.cross_val_score(clf_bag, x, y, cv=cv)
rez

array([0.86486486, 0.95628415, 0.91712707, 0.93333333, 0.91620112,
       0.97765363, 0.96648045, 0.91011236, 0.87570621, 0.92045455])

In [18]:
rez.mean()

0.9238217738251967

In [19]:
with open("task2.txt", "w") as file:
    file.write(str(rez.mean()))

# Задание 3

Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, а на d ** 0.5 случайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [20]:
clf_bag_8 = ensemble.BaggingClassifier(clf_tree, n_estimators=100, max_features=8)
rez = model_selection.cross_val_score(clf_bag_8, x, y, cv=cv)
rez

array([0.88648649, 0.95081967, 0.95027624, 0.9       , 0.94413408,
       0.94972067, 0.97206704, 0.97191011, 0.88700565, 0.91477273])

In [21]:
rez.mean()

0.9327192678770846

In [22]:
with open("task3.txt", "w") as file:
    file.write(str(rez.mean()))

# Задание 4

Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Какой параметр за это отвечает, можно понять из документации sklearn, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же 
d ** 0.5 признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [23]:
clf_tree_8 = tree.DecisionTreeClassifier(max_features=8)
clf_bag_tree_8 = ensemble.BaggingClassifier(clf_tree_8, n_estimators=100)
rez = model_selection.cross_val_score(clf_bag_tree_8, x, y, cv=cv)
rez

array([0.91351351, 0.9726776 , 0.93370166, 0.94444444, 0.96089385,
       0.97765363, 0.97206704, 0.95505618, 0.93220339, 0.95454545])

In [24]:
rez.mean()

0.9516756760335845

In [25]:
with open("task4.txt", "w") as file:
    file.write(str(rez.mean()))

# Задание 5

Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.

На основе наблюдений выпишите через пробел номера правильных утверждений из приведенных ниже в порядке возрастания номера (это будет ответ в п.5)

1. Случайный лес сильно переобучается с ростом количества деревьев

2. При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев

3. С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.

4. При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.

5. При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.

6. При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.

7. При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).

In [26]:
# Решил в уме:)
with open("task5.txt", "w") as file:
    file.write("2 3 4 7")